In [1]:
from datetime import datetime

from tqdm import tqdm
import pandas as pd
import plotly.express as px

from vnpy.trader.database import get_database
from vnpy.trader.constant import Interval, Exchange

# 数据处理

In [ ]:
db = get_database()

In [ ]:
bars = db.load_bar_data(
    symbol="i888",
    exchange=Exchange.DCE,
    interval=Interval.MINUTE,
    start=datetime(2010, 1, 1),
    end=datetime.now()
)

In [ ]:
df = pd.DataFrame.from_dict(b.__dict__ for b in bars)
df.head()

In [ ]:
df.drop(labels=["gateway_name", "interval", "symbol", "exchange", "vt_symbol"], axis=1, inplace=True)

In [ ]:
df.to_feather("i888.fth")

# 遍历回测

In [2]:
df = pd.read_feather("i888.fth")
df.set_index("datetime", inplace=True)

In [3]:
df = df[df.index > "2022-1-1"]

In [4]:
df["fast_ma"] = df["close_price"].rolling(20).mean()
df["slow_ma"] = df["close_price"].rolling(100).mean()
df["signal"] = (df["fast_ma"] - df["slow_ma"]).fillna(0)

In [18]:
def run_backtesting(
    df: pd.DataFrame,
    window: int = 10000,
    tp_percent: float = 0.05,
    sl_percent: float = 0.05,
    quantile: float = 0.2,
    capital: int = 1_000_000,
    commission: float = 3 / 10000,
    show_progress: bool = True
) -> None:
    """执行回测任务"""
    # 滚动窗口计算分位数
    df["long_entry"] = df["signal"].rolling(window).quantile(1 - quantile)
    df["short_entry"] = df["signal"].rolling(window).quantile(quantile)

    # 初始化状态变量
    pos = 0         # 持仓
    fee = 0         # 交易成本
    long_sl = 0     # 多头止损
    long_tp = 0     # 多头止盈
    short_sl = 0    # 空头止损
    short_tp = 0    # 空头止盈

    pos_result = {}     # 持仓结果字典
    fee_result = {}     # 交易成本字典

    # 遍历计算仓位
    it = df.rolling(window)
    if show_progress:
        it = tqdm(it, total=(len(df)))

    for v in it:
        # 检查数据长度
        if len(v.index) < window:
            continue

        # 提取数据
        row = v.iloc[-1, :]

        last_ix = row.name
        last_signal = row.signal
        last_price = row.close_price
        long_entry = row.long_entry
        short_entry = row.short_entry

        # 记录当前仓位
        pos_result[last_ix] = pos
        fee_result[last_ix] = fee

        # 重置交易成本
        fee = 0

        # 判断开仓
        if not pos:
            if last_signal >= long_entry:
                pos = int(round(capital / last_price))
                fee = abs(pos * last_price * commission)

                long_sl = last_price * (1 - sl_percent)
                long_tp = last_price * (1 + tp_percent)
            elif last_signal <= short_entry:
                pos = -int(round(capital / last_price))
                fee = pos * last_price * commission

                short_sl = last_price * (1 + sl_percent)
                short_tp = last_price * (1 - tp_percent)
        # 多头平仓
        elif pos > 0:
            if last_price >= long_tp or last_price <= long_sl:
                fee = abs(pos * last_price * commission)

                pos = 0
                long_sl = 0
                long_tp = 0
        # 空头平仓
        elif pos < 0:
            if last_price <= short_tp or last_price >= short_sl:
                fee = abs(pos * last_price * commission)

                pos = 0
                short_sl = 0
                short_tp = 0                
    
    # 统计盈亏结果
    df["pos"] = pd.Series(pos_result)
    df["fee"] = pd.Series(fee_result)
    df["change"] = (df["close_price"] - df["close_price"].shift(1)).fillna(0)
    df["pnl"] = df["change"] * df["pos"] - df["fee"]

    df["signal_nav"] = df["pnl"].cumsum() / capital + 1
    df["index_nav"] = df["close_price"] / df["close_price"].iat[0]

    return df

In [19]:
df = run_backtesting(df)

100%|██████████| 82200/82200 [00:08<00:00, 10138.92it/s]


In [21]:
fee = df["fee"].fillna(0)
fee[fee != 0]

datetime
2022-02-21 13:54:00+08:00    300.03555
2022-03-01 09:28:00+08:00    315.61860
2022-03-01 09:29:00+08:00    300.02700
2022-03-02 10:30:00+08:00    315.09900
2022-03-02 10:31:00+08:00    299.98080
                               ...    
2022-11-25 21:05:00+08:00    300.06480
2022-12-02 22:46:00+08:00    315.43200
2022-12-02 22:48:00+08:00    299.90400
2022-12-09 13:56:00+08:00    315.07200
2022-12-09 13:57:00+08:00    300.04800
Name: fee, Length: 145, dtype: float64